# Azure AI Foundry Agent Service - Connected Agents

[Connected Agents](https://learn.microsoft.com/en-us/azure/ai-foundry/agents/how-to/connected-agents?pivots=python) 機能を用いて、これまで作成した Foundry Agents を連携させたマルチエージェントシステムを構築するノートブックです。

## 概要

Connected Agents は Azure AI Foundry Agent Service 上のエージェントを専門的な役割に分解し、カスタムオーケストレーターや手動のルーティングロジックを必要とせずに、マルチエージェントシステムを構築できます。

### Connected Agentsの主な特徴

1. **ワークフローデザインの簡素化**: 複雑なタスクを専門エージェント間で分割し、複雑さを軽減して明確性を向上
2. **カスタムオーケストレーション不要**: メインエージェントが自然言語を使ってタスクをルーティングし、ハードコードされたロジックが不要
3. **拡張性**: 新しいConnected Agents（翻訳やリスク評価など）をメインエージェントを変更せずに追加可能
4. **信頼性とトレーサビリティの向上**: 各エージェントに集中的な責任を割り当て、デバッグの簡素化と監査性の向上
5. **柔軟なセットアップオプション**: Foundryポータルのノーコードインターフェースまたは Python SDK を使った プログラマティックな設定


### Connected Agents の主な制限

1. **ローカル関数は呼び出せない**
    * **制限内容**：Connected Agents 機能を使って別のエージェントをツールとして追加した場合、その接続されたエージェント側で定義した関数（Pythonのローカル関数）をFunction Tool経由で直接呼び出すことはできません。
    * **推奨される方法**：どうしても外部の処理を呼び出したい場合は、「OpenAPIツール」や「Azure Functions」を使って、API経由で実装することが推奨されています。

2. **出典（引用）の引き継ぎが保証されない**
    * **制限内容**：Connected Agents経由で検索や回答をした際、接続されたエージェントが生成した引用情報（例えばドキュメントの出典や根拠）が、親エージェントに必ず渡るとは限りません。
    * **対応策（完全ではない）**：プロンプトの工夫（Prompt Engineering）や、使うAIモデルを変えることで出典が返りやすくする方法もありますが、必ず引用が親エージェントに引き継がれる保証はありません。

# ライブラリのインポート

必要なPythonライブラリとAzure AI Foundry SDKをインポートします。

In [19]:
import os
import time
import json
import datetime
import zoneinfo

import requests

from dotenv import load_dotenv, find_dotenv

from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.agents.models import (
    MessageTextContent,
    ListSortOrder,
    McpTool,
    MCPToolDefinition,
    RequiredMcpToolCall,
    SubmitToolApprovalAction,
    ToolApproval,
    CodeInterpreterTool,
    FunctionTool,
    ToolSet,
    FilePurpose,
    FileSearchTool,
    RunAdditionalFieldList,
    RunStepFileSearchToolCall,
    RunStepToolCallDetails,
    ConnectedAgentTool
)


# 環境変数の取得

Azure AI Foundryへの接続に必要な設定情報を環境変数から取得します。

- `PROJECT_ENDPOINT`: Azure AI Foundryプロジェクトのエンドポイント
- `AZURE_DEPLOYMENT_NAME`: 使用するAIモデルのデプロイメント名
- `FOUNDRY_CODE_INTERPRETER_AGENT_ID`：先ほど作成した Code Interpreter Agent
- `FOUNDRY_FILE_SEARCH_AGENT_ID`：先ほど作成した File Search Agent

これらの値は事前に.envファイルに設定されている必要があります。

In [20]:
load_dotenv(override=True)

PROJECT_ENDPOINT=os.getenv("PROJECT_ENDPOINT")
AZURE_DEPLOYMENT_NAME=os.getenv("AZURE_DEPLOYMENT_NAME")
FOUNDRY_CODE_INTERPRETER_AGENT_ID=os.getenv("FOUNDRY_CODE_INTERPRETER_AGENT_ID")
FOUNDRY_FILE_SEARCH_AGENT_ID=os.getenv("FOUNDRY_FILE_SEARCH_AGENT_ID")

# クライアントの初期化

Azure AI FoundryのProject ClientとAgents Clientを初期化します。

- **AIProjectClient**: Azure AI Foundryプロジェクトへの接続を管理
- **DefaultAzureCredential**: Entra ID認証を自動的に処理
- **agents_client**: エージェントの作成、取得、実行を管理

In [21]:
# AI Project Client を初期化
project_client = AIProjectClient(
    endpoint=PROJECT_ENDPOINT,
    credential=DefaultAzureCredential()
)

# AgentClient の作成
agents_client = project_client.agents

# ユーティリティ関数

### agent_run_outputs関数
- スレッド内のメッセージ一覧を取得・表示
- 画像コンテンツがある場合は保存・表示
- ツール呼び出し情報の詳細表示（Run Stepsから取得）

In [22]:
def agent_run_outputs(thread_id, agents_client, target_dir="./output_images", show_tool_calls=True, run_id=None):
    """
    指定したスレッドIDのRun実行結果（テキスト・画像・ツール呼び出し）をNotebook上に表示＆画像は保存。
    
    Args:
        thread_id: スレッドID
        agents_client: エージェントクライアント
        target_dir: 画像保存ディレクトリ
        show_tool_calls: ツール呼び出し情報を表示するかどうか
        run_id: 特定のRunのツール呼び出し情報を表示する場合のRun ID
    """
    from IPython.display import Image, display
    from azure.ai.agents.models import RunStepToolCallDetails, RunStepFunctionToolCall, RunStepMessageCreationDetails
    
    messages = agents_client.messages.list(thread_id=thread_id, order=ListSortOrder.ASCENDING)
    os.makedirs(target_dir, exist_ok=True)

    # メッセージの重複防止
    displayed_message_ids = set()
    
    # メッセージの表示
    for message in messages:
        # メッセージの重複チェック
        if message.id in displayed_message_ids:
            continue
        displayed_message_ids.add(message.id)
        
        print(f"\n{'='*60}")
        print(f"MESSAGE ROLE: {message.role.upper()}")
        print(f"MESSAGE ID: {message.id}")
        print(f"{'='*60}")
        
        # テキスト出力
        if message.text_messages:
            for txt in message.text_messages:
                print(f"{txt.text.value}")
        
        # 画像出力
        if hasattr(message, "image_contents") and message.image_contents:
            print(f"\n[IMAGES]")
            for image_content in message.image_contents:
                file_id = image_content.image_file.file_id
                file_name = f"{file_id}_image_file.png"

                agents_client.files.save(
                    file_id=file_id,
                    file_name=file_name,
                    target_dir=target_dir
                )
                print(f"  Saved image: {file_name}")
                display(Image(filename=f"{target_dir}/{file_name}"))
    
    # ツール呼び出し情報の表示（Run Stepsから取得）
    if show_tool_calls and run_id:
        print(f"\n{'='*60}")
        print(f"RUN STEPS INFORMATION (RUN ID: {run_id})")
        print(f"{'='*60}")
        
        try:
            # Run Stepsを取得（デフォルトは新しい順なので、古い順に並び替え）
            run_steps = agents_client.run_steps.list(thread_id=thread_id, run_id=run_id)
            run_steps_list = list(run_steps)
            run_steps_list.reverse()  # 実行順序に並び替え（STEP1から順番に）
            
            # 重複防止のためのセット
            displayed_step_ids = set()
            
            print(f"Total Run Steps: {len(run_steps_list)}")
            
            # 全てのrun stepsを実行順序で表示
            for step_num, run_step in enumerate(run_steps_list, 1):
                # 重複チェック
                if run_step.id in displayed_step_ids:
                    print(f"[STEP {step_num}] - SKIPPED (Duplicate Step ID: {run_step.id})")
                    continue
                displayed_step_ids.add(run_step.id)
                
                print(f"\n[STEP {step_num}] - {run_step.type}")
                print(f"  Step ID: {run_step.id}")
                print(f"  Status: {run_step.status}")
                
                # Message Creation Step
                if isinstance(run_step.step_details, RunStepMessageCreationDetails):
                    print(f"  Message Creation Step")
                    if hasattr(run_step.step_details.message_creation, 'message_id'):
                        print(f"  Message ID: {run_step.step_details.message_creation.message_id}")
                
                # Tool Calls Step
                elif isinstance(run_step.step_details, RunStepToolCallDetails):
                    print(f"  Tool Calls Step - {len(run_step.step_details.tool_calls)} tool(s)")
                    
                    for tool_num, tool_call in enumerate(run_step.step_details.tool_calls, 1):
                        print(f"\n    [TOOL CALL {tool_num}]")
                        print(f"    Tool Type: {tool_call.type}")
                        print(f"    Tool Call ID: {tool_call.id}")
                        
                        # Function Tool Call の詳細
                        if isinstance(tool_call, RunStepFunctionToolCall):
                            print(f"    Function Name: {tool_call.function.name}")
                            print(f"    Function Arguments: {tool_call.function.arguments}")
                            # 関数の実行結果を表示（利用可能な場合）
                            if hasattr(tool_call.function, 'output') and tool_call.function.output:
                                print(f"    Function Output: {tool_call.function.output}")
                            elif hasattr(tool_call.function, 'outputs') and tool_call.function.outputs:
                                print(f"    Function Outputs: {tool_call.function.outputs}")
                            elif hasattr(tool_call.function, 'result') and tool_call.function.result:
                                print(f"    Function Result: {tool_call.function.result}")
                        
                        print(f"    {'-'*30}")
                
                # その他のステップタイプ
                else:
                    print(f"  Step Type: {type(run_step.step_details).__name__}")
                
                print(f"  Created At: {run_step.created_at}")
                if hasattr(run_step, 'completed_at') and run_step.completed_at:
                    print(f"  Completed At: {run_step.completed_at}")
                
                print(f"  {'='*50}")
                
        except Exception as e:
            print(f"Error retrieving run steps: {e}")
            print(f"Run ID: {run_id}, Thread ID: {thread_id}")

# Connected Agents の定義

Connected Agentsシステムで使用するエージェントを定義・取得します。

### 既存エージェントの取得
01-04のチュートリアルで作成した専門エージェントをEnvironment Variableから取得：
- **Code Interpreter Agent**: 計算・データ分析・グラフ生成
- **File Search Agent**: ドキュメント検索・カスタマーサポート

### ConnectedAgentToolの作成
各エージェントを`ConnectedAgentTool`として定義し、メインエージェントが呼び出せるツールに変換します。各ツールには以下が必要：
- **id**: 接続するエージェントのID
- **name**: 関数呼び出し用の名前（機械可読）
- **description**: エージェントの説明

In [23]:
# エージェントの取得
code_interpreter_agent = agents_client.get_agent(agent_id=FOUNDRY_CODE_INTERPRETER_AGENT_ID)
file_search_agent = agents_client.get_agent(agent_id=FOUNDRY_FILE_SEARCH_AGENT_ID)

# ConnectedAgentTool の作成
code_interpreter_agent_tool = ConnectedAgentTool(
    id=code_interpreter_agent.id,
    name="code_interpreter_agent",
    description=(
        "Code Interpreter を利用して、計算や図表の出力に特化した分析アシスタントです。"
    ),
)

file_search_agent_tool = ConnectedAgentTool(
    id=file_search_agent.id,
    name="file_search_agent",
    description=(
        "File Search を利用して、ファイルの検索や内容の取得を行うことができます。"
    ),
)

# トリアージエージェントの作成

親エージェントとして機能する **Triage Agent** を作成します。

- **Triage Agentの役割**：
    ユーザーの質問や相談内容を分析し、適切な専門エージェントにタスクを振り分けます：
- **Connected Agentsの設定**：
    `tools`パラメータに`ConnectedAgentTool.definitions`を指定することで、他のエージェントをツールとして利用可能になります。

In [24]:
triage_agent  = agents_client.create_agent(
    model=AZURE_DEPLOYMENT_NAME,
    name="triage_agent ",
    instructions=(
        "あなたは振り分けエージェントです。ユーザーの質問にあわせて適切な専門エージェントにタスクを割り振ります。"
        " - file_search_agent: カスタマーサポート担当で、社内のドキュメントを検索し、ユーザーの質問に答えます。"
        " - code_interpreter_agent: 計算に特化した分析アシスタントです。"
    ),
    tools=[
        code_interpreter_agent_tool.definitions[0],
        file_search_agent_tool.definitions[0],
    ],
)
print(f"Created Agent. AGENT_ID: {triage_agent.id}")


Created Agent. AGENT_ID: asst_hD8o34zn2INxishHexpUYEvo


# スレッドの作成

エージェントとの対話を管理するスレッドを作成します。

In [25]:
# Thread の作成
thread = agents_client.threads.create()
print(f"Created Thread_1. THREAD_1_ID: {thread.id}")

Created Thread_1. THREAD_1_ID: thread_DiwScO4xzO7QFiOzARLlJ96Q


# ユーザーメッセージの追加

スレッドにユーザーからの質問やリクエストを追加します。

In [26]:
# メッセージの追加
user_message = (
    "パソコンの保障プランについてどんなプランがあるか教えて。"
)

message = agents_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=user_message,
)

print(f"Added Message. MESSAGE_ID: {message.id}")

Added Message. MESSAGE_ID: msg_5eH0bD8KCL3Mu4okn2Qcu0Z0


# Run の実行

エージェントにタスクを実行させ、結果を取得します。

### Run1

In [27]:
run_1 = agents_client.runs.create_and_process(
    thread_id=thread.id,
    agent_id=triage_agent.id,
)

if run_1.status == "failed":
    print(f"Run failed: {run_1.last_error}")
else:
    agent_run_outputs(thread.id, agents_client, show_tool_calls=True, run_id=run_1.id)


MESSAGE ROLE: USER
MESSAGE ID: msg_5eH0bD8KCL3Mu4okn2Qcu0Z0
パソコンの保障プランについてどんなプランがあるか教えて。

MESSAGE ROLE: ASSISTANT
MESSAGE ID: msg_diJUmNAYYVVHyMNwIbAkCemU
パソコンの保障プランには、主に以下のような種類と内容があります。

■ プランの種類
- 1年プラン：10,000円（税込）、保証期間1年
- 3年プラン：20,000円（税込）、保証期間3年
- 5年プラン：30,000円（税込）、保証期間5年

■ 主な保障内容
- 故障時の修理費用を全額カバー（一部例外あり）
- 修理期間中は代替機を最短3営業日で提供
- 論理障害のデータ復旧サービス（成功率約90%）
- 24時間365日の専用サポート窓口（電話・Web対応）

■ 対象パソコン
- Microsoft Surfaceシリーズ全モデル
- Microsoftブランドのデスクトップ・ノートPC
- 個人・法人問わず利用可能

■ 保証の対象外
- 水没・火災・落下等の損傷
- 盗難・紛失・自然災害による破損
- ウイルス感染やソフトウェア起因のトラブル
- 改造や分解、故意・過失による破損
- 物理障害のデータ復旧

■ その他
- 申込期限は、購入時または購入日から30日以内
- 1年・3年プランは1回のみ最大+1年延長可能、5年プランは延長不可
- 支払いはクレジットカードまたは銀行振込（一括のみ）

詳細な比較表やFAQもご案内可能ですので、必要な際はお申し付けください 。

RUN STEPS INFORMATION (RUN ID: run_ih5U3SAR0SbdUqNY5LBJ0KLF)
Total Run Steps: 2

[STEP 1] - RunStepType.TOOL_CALLS
  Step ID: step_1c1gLRguBMQqKTMkew31iiwv
  Status: RunStepStatus.COMPLETED
  Tool Calls Step - 1 tool(s)

    [TOOL CALL 1]
    Tool Type: connected_agent
    Tool Call ID: cal

### Run2

In [28]:
# メッセージの追加
user_message = (
    "パソコン10万円を購入し、3年目で壊れる確率が最も高いと仮定した場合、"
    "3年プランと5年プランのどちらがコスパが良いか、年間あたりの保証コストも計算して説明してください。"
)
message = agents_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=user_message,
)

# Run の実行
run_2 = agents_client.runs.create_and_process(
    thread_id=thread.id,
    agent_id=triage_agent.id,
)

if run_2.status == "failed":
    print(f"Run failed: {run_2.last_error}")
else:
    agent_run_outputs(thread.id, agents_client, show_tool_calls=True, run_id=run_2.id)


MESSAGE ROLE: USER
MESSAGE ID: msg_5eH0bD8KCL3Mu4okn2Qcu0Z0
パソコンの保障プランについてどんなプランがあるか教えて。

MESSAGE ROLE: ASSISTANT
MESSAGE ID: msg_diJUmNAYYVVHyMNwIbAkCemU
パソコンの保障プランには、主に以下のような種類と内容があります。

■ プランの種類
- 1年プラン：10,000円（税込）、保証期間1年
- 3年プラン：20,000円（税込）、保証期間3年
- 5年プラン：30,000円（税込）、保証期間5年

■ 主な保障内容
- 故障時の修理費用を全額カバー（一部例外あり）
- 修理期間中は代替機を最短3営業日で提供
- 論理障害のデータ復旧サービス（成功率約90%）
- 24時間365日の専用サポート窓口（電話・Web対応）

■ 対象パソコン
- Microsoft Surfaceシリーズ全モデル
- Microsoftブランドのデスクトップ・ノートPC
- 個人・法人問わず利用可能

■ 保証の対象外
- 水没・火災・落下等の損傷
- 盗難・紛失・自然災害による破損
- ウイルス感染やソフトウェア起因のトラブル
- 改造や分解、故意・過失による破損
- 物理障害のデータ復旧

■ その他
- 申込期限は、購入時または購入日から30日以内
- 1年・3年プランは1回のみ最大+1年延長可能、5年プランは延長不可
- 支払いはクレジットカードまたは銀行振込（一括のみ）

詳細な比較表やFAQもご案内可能ですので、必要な際はお申し付けください 。

MESSAGE ROLE: USER
MESSAGE ID: msg_wjaZh0uXWBWJansA57CGAPaw
パソコン10万円を購入し、3年目で壊れる確率が最も高いと仮定した場合、3年プランと5年プランのどちらがコスパが良いか、年間あたりの保証コストも計算して説明してください。

MESSAGE ROLE: ASSISTANT
MESSAGE ID: msg_V3WePrMPwrQOswebBHDzpDXJ
ご希望のケースに基づく計算と解説は以下の通りです。

---

■ 3年目で壊れるパターンのコスト比較

- 3年プラン：20,000円（購入から3年目まで保証

# トレースの確認

[Azure AI Foundry Portal](https://ai.azure.com/?cid=learnDocs) でエージェントの実行トレースを確認してみましょう。

# Agent ID を .env ファイルに保存

作成したFile SearchエージェントのIDを永続化し、他のノートブックで再利用できるようにします。

In [29]:
# 変数の定義
agent_env_key = "FOUNDRY_TRIAGE_AGENT_ID"
agent_env_value = triage_agent.id

# .envファイルのパスを自動探索
env_path = find_dotenv()  # 見つからなければ''を返す
if not env_path:
    raise FileNotFoundError(".envファイルが見つかりませんでした。")

# AGENT_ID を .env ファイルに追記
with open(env_path, "a", encoding="utf-8") as f:
    f.write(f'\n{agent_env_key}="{agent_env_value}"')

print(f'.envファイルに {agent_env_key}=\"{agent_env_value}\" を追記しました。')


.envファイルに FOUNDRY_TRIAGE_AGENT_ID="asst_hD8o34zn2INxishHexpUYEvo" を追記しました。
